# How to use Table storage from Python

This guide shows you how to perform common scenarios by using the Azure Table storage service. The samples are written in Python and use the [Python Azure Storage package](https://pypi.python.org/pypi/azure-storage). The covered scenarios include creating and deleting a table, in addition to inserting and querying entities in a table.

<div style="background-color: #eee;display: block; margin-top: 5px; position: relative; border-left-width: 10px; border-left-style: solid; border-left-color: #89c402; padding: 20px 15px 10px;"><h5 style="margin: 0 0 8px; font-weight: 400; font-size: 18px; line-height: 1.2em; color: #505050;">Important!</h5><p>The code sections in this document can be ran by clicking on a code section, then using the run icon on the toolbar at the top of the page. You can also modify the code to add your own values.</p><p>Some code sections are meant to be ran in a specific order. For example, the code section that creates a table must be ran before the code that performs operations on the table. These code sections will have a note before them, similar to this note.</p></div>

## What is the Table Service

The Azure Table storage service stores large amounts of
structured data. The service is a NoSQL datastore which accepts
authenticated calls from inside and outside the Azure cloud. Azure
tables are ideal for storing structured, non-relational data. Common
uses of the Table service include:

- Storing TBs of structured data capable of serving web scale
 applications
- Storing datasets that don't require complex joins, foreign keys, or
 stored procedures and can be denormalized for fast access
- Quickly querying data using a clustered index
- Accessing data using the OData protocol and LINQ queries with WCF
 Data Service .NET Libraries

You can use the Table service to store and query huge sets of
structured, non-relational data, and your tables will scale as demand
increases.

## Table Service Concepts

The Table service contains the following components:

![Table service components](http://larryfrstore.blob.core.windows.net/images/table1.png)

- **URL format:** Code addresses tables in an account using this
 address format: 
 http://`<storage account>`.table.core.windows.net/`<table>` 
 
 You can address Azure tables directly using this address with the
 OData protocol. For more information, see [OData.org](http://www.odata.org/)

- **Storage Account:** All access to Azure Storage is done
 through a storage account. See [Azure Storage Scalability and Performance Targets](https://azure.microsoft.com/en-us/documentation/articles/storage-scalability-targets) for details about storage account capacity.

- **Table**: A table is a collection of entities. Tables don't enforce
 a schema on entities, which means a single table can contain
 entities that have different sets of properties. The number of tables that a 
	storage account can contain is limited only by the 
 storage account capacity limit.

- **Entity**: An entity is a set of properties, similar to a database
 row. An entity can be up to 1MB in size.

- **Properties**: A property is a name-value pair. Each entity can
 include up to 252 properties to store data. Each entity also has 3
 system properties that specify a partition key, a row key, and a
 timestamp. Entities with the same partition key can be queried more
 quickly, and inserted/updated in atomic operations. An entity's row
 key is its unique identifier within a partition.

## Create an Azure storage account

The easiest way to create your first Azure storage account is by using the [Azure Management Portal](https://manage.windowsazure.com). To learn more, see [Create a storage account](https://azure.microsoft.com/en-us/documentation/articles/storage-create-storage-account.md#create-a-storage-account).

You can create an Azure storage account by using [Azure PowerShell](https://azure.microsoft.com/en-us/documentation/articles/storage-powehrshell-guide-full) or [Azure CLI](https://azure.microsoft.com/en-us/documentation/articles/storage-azure-cli.md), or by using the [Azure Storage Resource Provider REST API](https://msdn.microsoft.com/library/azure/mt163683.aspx).

<div style="background-color: #eee;display: block; margin-top: 5px; position: relative; border-left-width: 10px; border-left-style: solid; border-left-color: #89c402; padding: 20px 15px 10px;"><h5 style="margin: 0 0 8px; font-weight: 400; font-size: 18px; line-height: 1.2em; color: #505050;">Note</h5><p>If you need to install Python or the [Python Azure package](https://pypi.python.org/pypi/azure), see the [Python installation guide](https://azure.microsoft.com/en-us/documentation/articles/python-how-to-install/).</p></div>

## Create a table

The **TableService** object lets you work with table services. The
following code creates a **TableService** object. Add the code near
the top of any Python file in which you wish to programmatically access Azure Storage.

Select the following cell, and then use the run button on the toolbar at the top of the page to create the __TableService__ and __Entity__ objects. This code also defines the `props` function, which is used to inspect objects returned from inserting entities into a table.

<div style="background-color: #eee;display: block; margin-top: 5px; position: relative; border-left-width: 10px; border-left-style: solid; border-left-color: #89c402; padding: 20px 15px 10px;"><h5 style="margin: 0 0 8px; font-weight: 400; font-size: 18px; line-height: 1.2em; color: #505050;">Important!</h5><p>The following code must be ran before any other code in this document.</p></div>

In [ ]:
from azure.storage.table import TableService, Entity
import inspect

def props(obj):
    pr = {}
    for name in dir(obj):
        value = getattr(obj, name)
        if not name.startswith('__') and not inspect.ismethod(value):
            pr[name] = value
    return pr

The following code creates a **TableService** object by using the storage account name and account key. 

<div style="background-color: #eee;display: block; margin-top: 5px; position: relative; border-left-width: 10px; border-left-style: solid; border-left-color: #89c402; padding: 20px 15px 10px;"><h5 style="margin: 0 0 8px; font-weight: 400; font-size: 18px; line-height: 1.2em; color: #505050;">Important!</h5><p>Replace `myaccount` and `mykey` in the following code with the Storage Account name and key for your storage account before running this code. This will allow the code in this document to create, and work with, an example table named `tasktable` in your Azure Storage Account.</p><p>This code must be ran before code sections that perform table operations, such as insert, update, or delete.</p></div> 

A value of `true` will be returned when the table has been created.

In [ ]:
table_service = TableService(account_name='MyAccount', account_key='MyKey')
table_service.create_table('tasktable')

## Add an entity to a table

To add an entity, first create a dictionary that defines your entity
property names and values. Note that for every entity, you must
specify **PartitionKey** and **RowKey**. These are the unique
identifiers of your entities. You can query these values much
faster than you can query your other properties. The system uses **PartitionKey** to
automatically distribute the table entities over many storage nodes.
Entities that have the same **PartitionKey** are stored on the same node. **RowKey** is the unique ID of the entity within the partition that it
belongs to.

To add an entity to your table, pass a dictionary object
to the **insert\_entity** method. The object returned from method will appear similar to the following:
```{'PartitionKey': 'tasksSeattle',
 'RowKey': '1',
 'Timestamp': datetime.datetime(2015, 10, 1, 19, 29, 12, 678675, tzinfo=tzlocal()),
 'description': 'Take out the trash',
 'etag': 'W/"datetime\'2015-10-01T19%3A29%3A12.6786752Z\'"',
 'priority': 200}```

In [ ]:
task = {'PartitionKey': 'tasksSeattle', 
        'RowKey': '1', 
        'description' : 'Take out the trash', 
        'priority' : 200}
entity = table_service.insert_entity('tasktable', task)
props(entity)

You can also pass an instance of the **Entity** class to the **insert\_entity** method.

In [ ]:
task = Entity()
task.PartitionKey = 'tasksSeattle'
task.RowKey = '2'
task.description = 'Wash the car'
task.priority = 100
entity = table_service.insert_entity('tasktable', task)
props(entity)

If you wish to add additional entities, modify one of the previous code examples used to insert an entity by changing the following values:

* RowKey: Must be a unique value
* description: The description for this task
* priority: The priority for this task

After changing the values, use run to run the modified code and insert the new entity.

## Update an entity

This code shows how to replace the old version of an existing entity
with an updated version. This operation will return an [eTag](https://en.wikipedia.org/wiki/HTTP_ETag) for the updated entity. For example, `{'etag': 'W/"datetime\'2015-10-01T14%3A55%3A55.5015416Z\'"'}`.

In [ ]:
task = {'description' : 'Take out the garbage', 'priority' : 250}
table_service.update_entity('tasktable', 'tasksSeattle', '1', task)

If the entity that is being updated does not exist, then the update operation will fail. If you want to store an entity regardless of whether it existed before, use **insert\_or\_replace_entity**.

In the following example, the first call will replace the existing entity. The second call will insert a new entity, since no entity with the specified **PartitionKey** and **RowKey** exists in the table.

This operation will also return an eTag.

In [ ]:
task = {'description' : 'Take out the garbage again', 'priority' : 250}
table_service.insert_or_replace_entity('tasktable', 'tasksSeattle', '1', task)

task = {'description' : 'Buy detergent', 'priority' : 300}
table_service.insert_or_replace_entity('tasktable', 'tasksSeattle', '3', task)

## Change a group of entities

Sometimes it makes sense to submit multiple operations together in a
batch to ensure atomic processing by the server. To accomplish that, you
use the **begin\_batch** method on **TableService** and then call the
series of operations as usual. When you do want to submit the
batch, you call **commit\_batch**. Note that all entities must be in the same partition in order to be changed as a batch. 

The example below adds two entities together in a batch.

In [ ]:
task10 = {'PartitionKey': 'tasksSeattle', 
          'RowKey': '10', 
          'description' : 'Go grocery shopping', 
          'priority' : 400}
task11 = {'PartitionKey': 'tasksSeattle', 
          'RowKey': '11', 
          'description' : 'Clean the bathroom', 
          'priority' : 100}
table_service.begin_batch()
table_service.insert_entity('tasktable', task10)
table_service.insert_entity('tasktable', task11)
table_service.commit_batch()

## Query for an entity

To query an entity in a table, use the **get\_entity** method by
passing **PartitionKey** and **RowKey**. 

The example below will retrieve the entity with a RowKey of `1` and a PartitionKey of `tasksSeattle`, then print the `description` and `priority` information it contains. If you have used the default values in previous code sections, the values returned will be `Take out the garbage again, 250`.

In [ ]:
task = table_service.get_entity('tasktable', 'tasksSeattle', '1')
print("{0}, {1}".format(task.description, task.priority))

## Query a set of entities

This example finds all tasks in Seattle based on **PartitionKey**. 

The example below will find all tasks where the PartitionKey is `Seattle`, then print the `description` and `priority`. If you have used the default values in previous code sections, you should see a list that begins with `Take out the garbage again, 250`.

In [ ]:
tasks = table_service.query_entities('tasktable', "PartitionKey eq 'tasksSeattle'")
for task in tasks:
	print("{0}, {1}".format(task.description, task.priority))

## Query a subset of entity properties

A query to a table can retrieve just a few properties from an entity.
This technique, called *projection*, reduces bandwidth and can improve
query performance, especially for large entities. Use the **select**
parameter and pass the names of the properties that you want to bring over
to the client.

<div style="background-color: #eee;display: block; margin-top: 5px; position: relative; border-left-width: 10px; border-left-style: solid; border-left-color: #89c402; padding: 20px 15px 10px;"><h5 style="margin: 0 0 8px; font-weight: 400; font-size: 18px; line-height: 1.2em; color: #505050;">Note</h5><p>The following snippet works only against the cloud
storage service. This is not supported by the storage
emulator.</p></div>

The query in the following code returns only the descriptions of
entities in the table. If you have used default values in the previous code sections, you should see a list that begins with `Take out the garbage`.

In [ ]:
tasks = table_service.query_entities('tasktable', 
                                     "PartitionKey eq 'tasksSeattle'", 
                                     'description')
for task in tasks:
	print(task.description)

## Delete an entity

You can delete an entity by using its partition and row key.

In [ ]:
table_service.delete_entity('tasktable', 'tasksSeattle', '1')

If you return to the previous code section that lists tasks, and run it again, you will notice that the entity for RowKey 1, `Take out the garbage` is no longer returned.

## Delete a table

The following code deletes a table from a storage account.

This will return a value of `True` when the table is deleted.

In [ ]:
table_service.delete_table('tasktable')

## Next steps

Now that you have learned the basics of Table storage, follow these links
to learn about more complex storage tasks:

-   See the MSDN reference [Azure Storage])(http://msdn.microsoft.com/library/azure/gg433040.aspx.
-   Visit the [Azure Storage Team blog])(http://blogs.msdn.com/b/windowsazurestorage/.

For more information, see also the [Python Developer Center](/develop/python/).